In [1]:
setwd("../..")
pacman::p_load(ggplot2, dplyr, stringr, igraph, reshape2, lme4, tidyr)
options(stringsAsFactors = FALSE)

In [2]:
color_map = c("#70ada1", "#f2c18f", "#f28f8f", "#a099c9")
names(color_map) = c('Immune','Specie','KEGG','Metabolome')

---
# Input

In [3]:
wgcna_result = readRDS("output/correlation/table/WGCNA_geneTrees.rds")
module_decode = read.csv("output/correlation/table/Module_decode.csv", header = T, row.names = 1)

In [4]:
abun_me = wgcna_result$abun_result$MEs
bioc_me = wgcna_result$bioc_result$MEs
immune_me = wgcna_result$immune_result$MEs
kegg_me = wgcna_result$kegg_result$MEs

In [5]:
abun_me = abun_me[,rownames(module_decode)[grepl('Specie',rownames(module_decode))]]
kegg_me = kegg_me[,rownames(module_decode)[grepl('KEGG',rownames(module_decode))]]
immune_me = immune_me[,rownames(module_decode)[grepl('Immune',rownames(module_decode))]]
bioc_me = bioc_me[,rownames(module_decode)[grepl('Metabolome',rownames(module_decode))]]

In [6]:
sample=Reduce(intersect, list(rownames(abun_me),rownames(bioc_me),rownames(kegg_me),rownames(immune_me)))
table = cbind(abun_me[sample,], bioc_me[sample,], kegg_me[sample,], immune_me[sample,])

In [7]:
meta_data = read.csv("data/metadata/Metadata_100322.csv", row.names = 1, header = T)
meta_data = meta_data[sample, ]

In [8]:
subgroup_list = readRDS(file = "codes/correlation/Subgroup_sample_list.RDS")
subgroup_list = sapply(subgroup_list, function(X){X[X %in% rownames(table)]})

In [9]:
corr_mat_list = lapply(subgroup_list, function(X){cor(table[X,], method = 'spearman')})

# Define the base line (the comparison group)

In [10]:
corr_rev_list = lapply(subgroup_list, function(X){cor(table[!(rownames(table)%in%X),], method = 'spearman')})

In [11]:
healthy_name_list = names(corr_mat_list)[grepl(pattern = "Healthy",x = names(corr_mat_list))]
patient_name_list = names(corr_mat_list)[grepl(pattern = "Patient|Long|Short|IBS|Infection",
                                               x = names(corr_mat_list))]
for (i in patient_name_list) {corr_rev_list[[i]] = corr_mat_list[['Healthy']]}
for (i in healthy_name_list) {corr_rev_list[[i]] = corr_mat_list[['Patient']]}

# Plot Healthy

In [12]:
set.seed(123)
mat = corr_mat_list$Healthy
network = graph_from_adjacency_matrix(abs(mat), weighted=T, mode="undirected", diag=F)
layout = layout_with_fr(network)

In [13]:
vertices = get.data.frame(network, what="vertices")
vertices$color = color_map[str_split_fixed(vertices$name, "\\.",2)[,1]]
vertices$size = sqrt(module_decode[vertices$name, 'size'])*2
vertices$name = module_decode[vertices$name, 'decode']

edges = get.data.frame(network, what="edges")
edge_color_map <- colorRampPalette(c("#2166ac", "#f7f7f7", "#b2182b"))(1000)
edge_color_mapping <- function(x) {
    edge_color_map[findInterval(x, seq(-0.3, 0.3, length.out = 1000), all.inside = TRUE)]
  } 

for (i in 1:nrow(edges)) {
    corr_i = mat[edges[i,'from'],edges[i,'to']]
    edges[i, 'corr'] = corr_i
    edges[i, 'color'] = edge_color_mapping(corr_i)
    edges[i, 'weight'] = corr_i * 6
    }

edges$color = adjustcolor(edges$color, alpha.f = 0.6)

pdf("output/correlation/figure/Network_Healthy.pdf", width=6, height=7)
plot(network,    
     layout=layout,
    vertex.size=vertices$size,
    vertex.color=vertices$color,
    vertex.label = vertices$name,
    vertex.label.cex=0.9,
    vertex.label.font=1,
    vertex.label.color="black",
    vertex.frame.color="grey",     
    edge.color=edges$color,
    edge.width=abs(edges$weight),
    edge.curved=0.2
    )
dev.off()

png 
  2

In [14]:
pdf("output/correlation/figure/Network_Healthy_legend.pdf", width = 1, height = 3, bg = "transparent")
edges = edges[order(edges$corr),]
par(mar=c(2.5,0.5,2.5,0.5))
barplot(edges$corr, 
        col = edges$color, 
        width = abs(edges$weight), 
        border = NA, 
        main = "Spearman\nCoefficient",
        horiz = TRUE)
dev.off()

png 
  2

# Plot Patient, compare to healthy

In [16]:
edge_color_map <- colorRampPalette(c("#2166ac", "#f7f7f7", "#b2182b"))(1000)

edge_color_mapping <- function(x, cutoff) {
  # Get the color based on the edge weight
  color <- edge_color_map[findInterval(x, seq(-cutoff, cutoff, length.out = 1000), all.inside = TRUE)]
  
  # Calculate alpha based on the absolute edge weight, scaled from 0 (transparent) to 1 (opaque)
  alpha <- abs(x) / cutoff
  alpha[alpha > 1] <- 1  # Ensure alpha doesn't exceed 1
  
  # Combine color with alpha
  rgb_color <- col2rgb(color) / 255  # Convert color to RGB
  rgb_color <- apply(rgb_color, 2, function(col) rgb(col[1], col[2], col[3], alpha, maxColorValue = 1))
  return(rgb_color)
}


In [17]:
plot_patient <- function(mat, base_line, network=network, layout=layout) {
  vertices = get.data.frame(network, what="vertices")
  vertices$color = color_map[str_split_fixed(vertices$name, "\\.",2)[,1]]
  vertices$size = sqrt(module_decode[vertices$name, 'size'])*2
  vertices$name = module_decode[vertices$name, 'decode']
    
  edges <- get.data.frame(network, what = "edges")
  
  # Initialize edges
  edges$color <- '#f7f7f7'
  edges$weight <- 0
  
  # Update edges and vertices based on conditions
  for (i in 1:nrow(edges)) {
    corr_i <- mat[edges[i, 'from'], edges[i, 'to']]
    base_i <- base_line[edges[i, 'from'], edges[i, 'to']]
    diff <- corr_i - base_i
    edges[i, 'diff'] = diff
    edges[i, 'color'] <- edge_color_mapping(diff, cutoff = 0.4)
    edges[i, 'weight'] <- diff * 6
    }
    
  # Plot the network
  plot(network,    
     layout=layout,
    vertex.size=vertices$size,
    vertex.color=vertices$color,
    vertex.label = vertices$name,
    vertex.label.cex=0.9,
    vertex.label.font=1,
    vertex.label.color="black",
    vertex.frame.color="grey",     
    edge.color=edges$color,
    edge.width=abs(edges$weight),
    edge.curved=0.2
    )  
    
  return(edges)
}

In [18]:
pdf("output/correlation/figure/Network_Patient.pdf", width=6, height=7)
edges = plot_patient(corr_mat_list[['Patient']], corr_rev_list[['Patient']], network, layout)
dev.off()

pdf("output/correlation/figure/Network_Patient_legend.pdf", width=2, height=3, bg = "transparent")
edges = edges[order(edges$diff),]
par(mar=c(2.5,0.5,2.5,0.5))
barplot(edges$diff, 
        col = edges$color, 
        width = abs(edges$weight), 
        border = NA, 
        main = "Coefficient\nDelta",
        horiz = TRUE)
dev.off()

png 
  2

png 
  2

# Plot by subgroups

In [19]:
plot_one_group <- function(mat, base_line, network=network, layout=layout, name) {
  # Get vertices and edges data frames from the network
  vertices <- get.data.frame(network, what = "vertices")
  edges <- get.data.frame(network, what = "edges")
  
  # Initialize vertices
  vertices$color <- 'grey'
  vertices$name <- ''
  vertices$size <- 8
  
  # Initialize edges
  edges$color <- '#f7f7f7'
  edges$weight <- mat[cbind(edges$from, edges$to)] * 3
  
  # Update edges and vertices based on conditions
  for (i in 1:nrow(edges)) {
    corr_i <- mat[edges[i, 'from'], edges[i, 'to']]
    base_i <- base_line[edges[i, 'from'], edges[i, 'to']]
    diff <- corr_i - base_i
    edges[i, 'diff']=diff
    if (abs(diff) > 0.3) {
      edges[i, 'color'] <- edge_color_mapping(diff, cutoff = 0.4)
      edges[i, 'weight'] <- diff * 6
      
      from_module <- str_split_fixed(edges[i, 'from'], "\\.", 2)[, 1]
      to_module <- str_split_fixed(edges[i, 'to'], "\\.", 2)[, 1]
      
      vertices[edges[i, 'from'], 'color'] <- color_map[from_module]
      vertices[edges[i, 'to'], 'color'] <- color_map[to_module]
      
      vertices[edges[i, 'from'], 'name'] <- module_decode[edges[i, 'from'], 'decode']
      vertices[edges[i, 'to'], 'name'] <- module_decode[edges[i, 'to'], 'decode']
        
      vertices[edges[i, 'from'], 'size'] <- 10
      vertices[edges[i, 'to'], 'size'] <- 10
    }
  }
  
  # Plot the network
  plot(network,
             layout = layout,
             vertex.size = vertices$size,
             vertex.label.cex = 0.9,
             vertex.label.font = 1,
             vertex.color = vertices$color,
             vertex.label = vertices$name,
             vertex.label.color = "black",
             vertex.frame.color = "transparent",
             edge.color = edges$color,
             edge.width = abs(edges$weight),
             edge.curved = 0.2
  )
 name = str_replace(name,"_"," ")
 title(main = paste0("\n", name), cex.main = 2)
    
 return(edges)
}

In [20]:
group_name = c('Young_Patient','Female_Patient','NormalWeight_Patient','IBS',
              'Elder_Patient','Male_Patient','OverWeight_Patient','Infection')

pdf("output/correlation/figure/Network_Patient_Subgroups.pdf", width=10, height=5)

par(mfrow=c(2, 4),    # 2x4 grid
    mar=c(0,2,0,2),  # c(bottom, left, top, right)
    oma=c(0,1,0,1))  # Outer margins
edges = data.frame()
for (name in group_name) {
    mat = corr_mat_list[[name]]
    base_line = corr_rev_list[[name]]
    edges_i = plot_one_group(mat, base_line, network, layout, name)
    edges = rbind(edges, edges_i)
}
dev.off()

pdf("output/correlation/figure/Network_Patient_Subgroups_legend.pdf", width=1.5, height=3, bg = "transparent")
edges = edges[order(edges$diff),]
par(mar=c(2.5,0.5,2.5,0.5))
barplot(edges$diff, 
        col = edges$color, 
        width = abs(edges$weight), 
        border = NA, 
        main = "Coefficient\nDelta",
        horiz = TRUE)
dev.off()

png 
  2

png 
  2

In [21]:
group_name = c('Young_Healthy','Female_Healthy','NormalWeight_Healthy',
               'Elder_Healthy','Male_Healthy','OverWeight_Healthy')

pdf("output/correlation/figure/Network_Healthy_Subgroups.pdf", width=8.5, height=6)

par(mfrow=c(2, 3),   
    mar=c(0,2,0,2),  # c(bottom, left, top, right)
    oma=c(0,1,0,1))  # Outer margins
edges = data.frame()
for (name in group_name) {
    mat = corr_mat_list[[name]]
    base_line = corr_rev_list[[name]]
    edges_i = plot_one_group(mat, base_line, network, layout, name)
    edges = rbind(edges, edges_i)
}
dev.off()

pdf("output/correlation/figure/Network_Healthy_Subgroups_legend.pdf", width=1.5, height=3, bg = "transparent")
edges = edges[order(edges$diff),]
par(mar=c(2.5,0.5,2.5,0.5))
barplot(edges$diff, 
        col = edges$color, 
        width = abs(edges$weight), 
        border = NA, 
        main = "Coefficient\nDelta",
        horiz = TRUE)
dev.off()

png 
  2

png 
  2

In [22]:
group_name = c('Short','Long')
pdf("output/correlation/figure/Network_Short_Long.pdf", width=8, height=4)

par(mfrow=c(1, 2),   
    mar=c(0,2,0,2),  # c(bottom, left, top, right)
    oma=c(0,1,0,1))  # Outer margins

for (name in group_name) {
    mat = corr_mat_list[[name]]
    base_line = corr_rev_list[[name]]
    plot_one_group(mat, base_line, network, layout, name)
}
dev.off()

png 
  2